# Predict Future Sales

My very first competition using fastai ML

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import datetime

C:\Users\tient\Anaconda3\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = "data/predictsales/"

In [4]:
train = pd.read_csv(f'{PATH}sales_train_v2.csv', low_memory=False, 
                     )

In [ ]:
#df_raw.date=df_raw.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
train.shape

In [ ]:
train.head(10)

In [5]:
items_cat = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [6]:
test.shape

(214200, 3)

In [7]:
[c for c in train.columns if c not in test.columns]

['date', 'date_block_num', 'item_price', 'item_cnt_day']

In [8]:
items_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
items.head()

In [ ]:
shops.head()

In [ ]:
test.head()

In [6]:
items['item_name_len'] = items['item_name'].map(len) #Lenth of Item Description
items['item_name_wc'] = items['item_name'].map(lambda x: len(str(x).split(' '))) #Item Description Word Count

In [7]:
items_cat['item_category_name_len'] = items_cat['item_category_name'].map(len)  #Lenth of Item Category Description
items_cat['item_category_name_wc'] = items_cat['item_category_name'].map(lambda x: len(str(x).split(' '))) #Item Category Description Word Count

In [8]:
shops['shop_name_len'] = shops['shop_name'].map(len)  #Lenth of Shop Name
shops['shop_name_wc'] = shops['shop_name'].map(lambda x: len(str(x).split(' '))) #Shop Name Word Count

In [9]:
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train = train.drop(['date','item_price'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})

In [10]:
#Monthly Mean
shop_item_monthly_mean = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})

In [11]:
train = pd.merge(train, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])

In [12]:
train = pd.merge(train,items, how='left', on='item_id')

In [13]:
train = pd.merge(train,items_cat, how='left', on='item_category_id')

In [14]:
train = pd.merge(train, shops, how='left', on='shop_id')

In [15]:
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean,item_name,item_category_id,item_name_len,item_name_wc,item_category_name,item_category_name_len,item_category_name_wc,shop_name,shop_name_len,shop_name_wc
0,0,0,32,1,2013,6.0,8.0,1+1,40,3,1,Кино - DVD,10,3,"!Якутск Орджоникидзе, 56 фран",29,4
1,0,0,33,1,2013,3.0,3.0,1+1 (BD),37,8,2,Кино - Blu-Ray,14,3,"!Якутск Орджоникидзе, 56 фран",29,4
2,0,0,35,1,2013,1.0,7.5,10 ЛЕТ СПУСТЯ,40,13,3,Кино - DVD,10,3,"!Якутск Орджоникидзе, 56 фран",29,4
3,0,0,43,1,2013,1.0,1.0,100 МИЛЛИОНОВ ЕВРО,40,18,3,Кино - DVD,10,3,"!Якутск Орджоникидзе, 56 фран",29,4
4,0,0,51,1,2013,2.0,2.5,100 лучших произведений классики (mp3-CD) (Dig...,57,52,6,Музыка - MP3,12,3,"!Якутск Орджоникидзе, 56 фран",29,4


In [16]:
# Preprocessing test file
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)
test = pd.merge(test, items, how='left', on='item_id')
test = pd.merge(test, items_cat, how='left', on='item_category_id')
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_name,item_category_id,item_name_len,item_name_wc,item_category_name,item_category_name_len,item_category_name_wc,shop_name,shop_name_len,shop_name_wc,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,"NHL 15 [PS3, русские субтитры]",19,30,5,Игры - PS3,10,3,"Вологда ТРЦ ""Мармелад""",22,3,0.0
1,1,5,5320,11,2015,34,0.000000,ONE DIRECTION Made In The A.M.,55,31,7,Музыка - CD локального производства,35,5,"Вологда ТРЦ ""Мармелад""",22,3,0.0
2,2,5,5233,11,2015,34,2.000000,"Need for Speed Rivals (Essentials) [PS3, русск...",19,56,8,Игры - PS3,10,3,"Вологда ТРЦ ""Мармелад""",22,3,0.0
3,3,5,5232,11,2015,34,1.000000,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,59,9,Игры - XBOX 360,15,4,"Вологда ТРЦ ""Мармелад""",22,3,0.0
4,4,5,5268,11,2015,34,0.000000,"Need for Speed [PS4, русская версия]",20,36,6,Игры - PS4,10,3,"Вологда ТРЦ ""Мармелад""",22,3,0.0


In [17]:
train.shape

(1609124, 17)

In [18]:
test.shape

(214200, 18)

In [19]:
col = [c for c in test.columns if c not in train.columns]; col

['ID']

In [20]:
test = test.drop(col, axis=1)

In [21]:
train_cats(train); train_cats(test)

In [22]:
# Proc_df
x,y,nas = proc_df(train, 'item_cnt_month', max_n_cat=6)
x_test, y_test, nas = proc_df(test, 'item_cnt_month', max_n_cat=6)

In [23]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 214200  # same as Kaggle's test set size
n_trn = len(train)-n_valid
X_train, X_valid = split_vals(x, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((1394924, 16), (1394924,), (214200, 16))

In [24]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    #result
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [25]:
y = np.log1p(y.clip(0.,20.))
y_train = np.log1p(y_train.clip(0.,20.))
y_valid = np.log1p(y_valid.clip(0.,20.))

In [26]:
set_rf_samples(100000)

In [31]:
# m = RandomForestRegressor(n_estimators=80, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

Wall time: 2.12 s
[0.3173341327567057, 0.34034605090268005, 0.54116222325717, 0.36889772229659856]


In [32]:
reset_rf_samples()

In [27]:
m = RandomForestRegressor(n_estimators = 160, min_samples_leaf =3, max_features = 0.5, n_jobs=-1)
%time m.fit(x, y)
print_score(m)

Wall time: 27.9 s
[0.29444112929576005, 0.2815698820419142, 0.6049768564058653, 0.5680527495700831]


In [28]:
prediction = m.predict(x_test).clip(0.,20.)

In [29]:
test = pd.read_csv(f'{PATH}test.csv')

In [30]:
my_submission = pd.DataFrame({'ID': test.ID, 'item_cnt_month': prediction})

In [31]:
my_submission.to_csv('sales_submission.csv', index=False)

# Naive implementation of Random Forest

## Just stick with the training set and nothing more

1st attempt - 1130 out of 1296 LMAO. Don't have any categorical datas yet. Will imrove in next versions

2nd attempt - 867 out of 1387. Still a lot of things to do! Keep going <:-P Next I'll try some interpretation and see if anything can be done to improve the model. Also need to speed up the training, the model takes more than 6 mins to train. Need to read more about Time Series.

In [ ]:
# fastai
# df, y, nas = proc_df(df_raw, 'item_cnt_month')

In [ ]:
col = [c for c in df_raw.columns if c not in ['item_cnt_month']]
#Validation Hold Out Month
x1 = df_raw[df_raw['date_block_num']<33]
y1 = np.log1p(x1['item_cnt_month'].clip(0.,20.))
x1 = x1[col]
x2 = df_raw[df_raw['date_block_num']==33]
y2 = np.log1p(x2['item_cnt_month'].clip(0.,20.))
x2 = x2[col]

reg = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=18)
reg.fit(x1,y1)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.))))

In [ ]:
reg.fit(df_raw[col],df_raw['item_cnt_month'].clip(0.,20.))
test['item_cnt_month'] = reg.predict(test[col]).clip(0.,20.)
test[['ID','item_cnt_month']].to_csv('submission.csv', index=False)